In [1]:
import data_preproc as dp
import correlation_cognitive_methylation as ccm

files = {'cpg_values': 'D:/unn/GSE52588/GSE52588_beta_fn.npz',
         'cpg_names': 'D:/unn/GSE52588/GSE52588_beta_fn.npz', 
         'data_samples': 'D:/unn/GSE52588/GSE52588_samples.txt',
         'cognitive_frame': 'D:/unn/GSE52588/DOWN_FENOTIPO_No4,8,12_PerCorrelazioni.tsv',
         'cpgs_annotations': 'D:/unn/GSE52588/cpgs_annotations.txt'} 
directory_research = 'D:/unn/down_syndrome_epigenetic'
cognitive_indicator = 'corsi'
alpha = 0.001

methylation_frame, cognitive_frame, indicator_folder = dp.data_preproc(files, directory_research, cognitive_indicator)
correlation_frame = ccm.correlation_cognitive_methylation(alpha, files, indicator_folder, cognitive_indicator, 
                                                      methylation_frame, cognitive_frame)

100%|█████████████████████████████████████████████████████| 422802/422802 [06:50<00:00, 1029.60it/s]
